# Analysis of Energy Consumption in a Fully Electric Domestic Residence

### Some lovely packages that we need to use

In [1]:
import pandas as pd
import numpy as np
import requests 
import datetime as dt
import json
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.dates
import csv
import glob
import random
import os
import sklearn
from sklearn import linear_model

### Setup and Run Simulations

In [ ]:
def PlotFullModel(directory,ashp_indoor_temps,ashp_sample_profiles,ashp_cops,dl_occs,ashp_rooms):
    
    # Select sample profile:
    ashp_external_temp_profile_tab = ashp_sample_profiles["external_temp_sample_1"][0]

    plotcolours = ["brown","purple","r","orange","g","olive","c","b"]
    fig, axs = plt.subplots(1,figsize=(24,8))
    ax1 = axs
    ax2 = ax1.twinx()

    for idx in range(len(ashp_cops)):
        ashp_indoor_temp = ashp_indoor_temps[idx]
        cop = ashp_cops[idx]
        occ = dl_occs[idx]
        rooms = ashp_rooms[idx]

        # ASHP: Load model:
        ashp_model_file = glob.glob(f"{directory}/Model/ModelsAndSampleData/ashp_model_cop_{cop}*.npz")[0]
        ashp_model_coeffs = np.load(file=ashp_model_file)["model_coeffs"]
        ashp_model = np.poly1d(ashp_model_coeffs)

        # scale for occupants:
        scale = 1
        if(rooms == 1):
            scale = 1/1.533
        elif(rooms == 2 or rooms == 3):
            scale = 1
        elif(rooms == 4 or rooms == 5):
            scale = 1.478

        
        ashp_time_format = "%Y-%m-%d %H:%M:%S"
        ashp_external_temp_profile_tab["timestamp"] = pd.to_datetime(ashp_external_temp_profile_tab["timestamp"],format=ashp_time_format)
        ashp_external_temp_profile = ashp_external_temp_profile_tab["external_air_temp"][::72].values

        ashp_external_temp_profile_cs = ashp_external_temp_profile_tab.set_index("timestamp")
        ashp_external_temp_profile_cs_day = ashp_external_temp_profile_cs.resample('D').mean()
        ashp_external_temp_profile_da = ashp_external_temp_profile_cs_day["external_air_temp"]
        ashp_temp_diff_profile = ashp_external_temp_profile_da - ashp_indoor_temp

        # average daily temp for year:
        ashp_average_daily_eat_year = ashp_external_temp_profile_da.mean()

        # timeseries:
        start_date = pd.Timestamp("01-01-2000").toordinal()
        end_date = pd.Timestamp("01-01-2001").toordinal()
        duration = len(ashp_temp_diff_profile)
        time_series = np.linspace(start_date,end_date,duration)
        time_series_ext = np.linspace(start_date,end_date,len(ashp_external_temp_profile))

        ashp_energy_series_predicted = ashp_model(ashp_temp_diff_profile)
        ashp_energy_series_predicted[ashp_energy_series_predicted < 0] = 0
        ashp_energy_series_predicted[np.isnan(ashp_energy_series_predicted) == True] = 0
        ashp_energy_series_predicted_scaled = ashp_energy_series_predicted*scale
        ashp_total_yearly_usage = ashp_energy_series_predicted_scaled.sum()
        print(f"ASHP yearly total usage for bedrooms = {rooms}, t = {ashp_indoor_temp}, cop = {cop}, and av EAT = {ashp_average_daily_eat_year}, = {ashp_total_yearly_usage} kWh")
        
        # Appliance Load: Load model:
        appl_model_file = glob.glob(f"{directory}/Model/ModelsAndSampleData/dl_model_{occ}_occupants*.npz")[0]
        appl_model_coeffs = np.load(file=appl_model_file)["model_coeffs"]
        appl_model = np.poly1d(appl_model_coeffs)
        
        # plt.scatter(occ_data["day"],occ_data["usage_kwh"],color=color,alpha=0.2)
        dl_time_series = np.linspace(1,365,365)
        dl_energy_series_predicted = appl_model(dl_time_series)
        dl_total_yearly_usage = dl_energy_series_predicted.sum()
        print(f"DL yearly total usage for occ = {dl_occs} = {dl_total_yearly_usage} kWh")



        total_energy_series_predicted = ashp_energy_series_predicted_scaled + dl_energy_series_predicted
        total_total_yearly_usage = total_energy_series_predicted.sum()
        print(f"Total yearly total usage = {total_total_yearly_usage} kWh")
        # print(len(total_energy_series_predicted))




        # plt.suptitle("Predicted ASHP Energy Consumption (kWh)")
        color = plotcolours[idx]
        l3 = ax2.plot(time_series,total_energy_series_predicted,color=color,alpha=0.4,label=f"Predicted Daily Average Consumption for COP = {cop}, IAT Set-point = {ashp_indoor_temp}, Bedrooms = {rooms}, and Number of Occupants = {occ}")
        # l2 = ax1.axhline(y=ashp_indoor_temp,color=color,alpha=0.5,linestyle="dashed",label=f"IAT Set-point = {ashp_indoor_temp}$^\circ$C")
    xticks = np.linspace(start_date,end_date,12)
    xticks = [round(n) for n in xticks]
    xlabels = [("01-"+str(dt.date.fromordinal(d)).split("-")[1]) for d in xticks]
    l1 = ax1.scatter(time_series_ext,ashp_external_temp_profile,color='b',alpha=0.5,s=2,label="Daily Average EAT for Profile 1")
    # l2 = ax1.scatter(time_series,temp_diff_profile,color='r',alpha=0.7,s=2,label="Temperature Difference ($^\circ$C)")
    # plt.title("Predicted ASHP Energy Consumption (kWh)")
    # plt.title(f"Predictors: ASHP COP = {cop}, Internal Temperature = {indoor_temp}($^\circ$C), Temperature Profile = {"external_temp_sample_1"}")
    ax1.set_xlabel("Date of Year dd-mm")
    ax1.set_ylabel("Temperature ($^\circ$C)")
    ax2.set_ylabel("Electricity Consumption (kWh)")
    ax1.set_xticks(ticks=xticks,labels=xlabels,rotation=90)
    ax1.set_ylim(top=30)
    ax2.set_ylim(top=75)
    # ax2.legend([l1,l2,l3],["Average Daily External Air Temperature","Indoor Set-point Temperature","Electrical Energy Consumption (kWh)"])
    ax1.legend(loc=2)
    ax2.legend(loc=1)

def CalculateHomeEnergyProfile(directory,user_input,sample_profiles):
    (cops,temps,occupants,rooms) = user_input
    PlotFullModel(directory,temps,sample_profiles,cops,occupants,rooms)

# User inputs:
directory = "/URSS_ASHP_Home_Model" # The location where the top level file "*/ASHP_HOME_MODEL" is saved, including the filename.
cops = [4.0,2.0,3.0] # The ASHP COP from [2.0,2.5,3.0,3.5,4.0]
temps = [17.0,22.0,20] # The internal set-point temperature in degrees C
occupants = [2,5,3] # The number of home occupants
rooms = [1,5,2] # The number of rooms
user_input = (cops,temps,occupants,rooms)
# Sample inputs:
sample_profiles = pd.DataFrame()
sample_eat_profiles =  [pd.read_pickle(f"{directory}/Model/ModelsAndSampleData/sample_temperature_profiles/profile1_00118.pkl")]

sample_profiles["external_temp_sample_1"] = sample_eat_profiles
CalculateHomeEnergyProfile(directory,user_input,sample_profiles)